# Astronomical Coordinates

In [ ]:
import matplotlib.pyplot as plt

import numpy as np
import astropy.units as u

from astropy.coordinates import SkyCoord, Distance, Galactic
from astroplan.plots import plot_finder_image

from astropy.table import QTable
from astroquery.gaia import Gaia

## International Celestial Reference System (ICRS)

- The standard celestial reference system adopted by the International Astronomical Union (IAU). 
- Its origin is at the barycenter of the Solar System.
- The axes are fixed with respect to a set of distant extragalactic objects (mostly quasars).
- Sometimes referred to as "equatorial" or "J2000" coordinates.

## Declination (DEC or δ)

- Equivalent to terrestrial latitude. 
- Points north of the celestial equator have positive declinations, while those to the south have negative declinations. 
- Declination is expressed in degrees [°], arc-minutes [ ′ ], and arc-seconds [ ′′ ].
- 1° = 60′ = 3600′′

#### Example: DEC = +23° 52′ 12.12′′

## Right Ascension (RA or α) 

- Roughly equivalent to terrestrial longitude. 
- The units of right ascension are hours, minutes, seconds [hms].
- RA = 0h points to the Sun at the March equinox.
- 1 hour in RA = 15°, 24 hours in RA = 360°

#### Example: RA = 20h 23m 12.12s

---

### Target - M104 (NGC 4594)

- Right ascension: 12h 39m 59.4s
- Declination: -11° 37′ 23″

## `SkyCoord(Coord, Frame)`

In Astropy, the most common way of representing and working with sky coordinates is to use `SkyCoord()`.

In [ ]:
target_coords = SkyCoord('12h 39m 59.4s',
                         '−11d 37m 23s',
                         frame='icrs')
target_coords

In [ ]:
target_coords.ra

In [ ]:
target_coords.ra.hour

In [ ]:
target_coords.ra.hms

In [ ]:
target_coords.ra.degree

In [ ]:
target_coords.dec

In [ ]:
target_coords.dec.degree

In [ ]:
target_coords.ra.degree, target_coords.dec.degree

### Or you can use `SkyCoord.from_name()` and let [Simbad](https://simbad.u-strasbg.fr/simbad/) do all the work for you.

In [ ]:
target_coords = SkyCoord.from_name('M 104')
target_coords

In [ ]:
target_coords = SkyCoord.from_name('NGC 4594')
target_coords

In [ ]:
target_coords.get_constellation()

### Galactic Coordinates

In [ ]:
target_coords.galactic

In [ ]:
target_coords.galactic.l

In [ ]:
target_coords.galactic.b

---

# Other Stuff

In [ ]:
target_one = SkyCoord.from_name('Sirius A')
target_one

In [ ]:
target_two = SkyCoord.from_name('Sirius B')
target_two

In [ ]:
target_sep = target_one.separation(target_two)

target_sep

In [ ]:
target_sep.degree

In [ ]:
target_sep.radian

In [ ]:
target_sep.arcsecond

---

# Gaia - Stars in the Future

A big part of the Gaia dataset we have not looked at is the data on the motion of stars through space.  

<p>
    <img src="https://uwashington-astro300.github.io/A300_images/ProperMotion.png" width = "400">
</p>

Proper motion is the measure of the observed changes in the apparent places of stars, as seen from the center of mass of the Solar System, compared to the more distant stars. Knowledge of the proper motion, distance, and radial velocity allows calculations of an object's motion through space.

In the `gaia_source_lite` database, the proper motion data is found in the columns:

```
pmra                        Proper motion in right ascension direction
pmdec                       Proper motion in declination direction
radial_velocity             Radial velocity
```

In [ ]:
moving_star_coords = SkyCoord('17h 57m 48.49847s',
                              '+04d 41m 36.1139',
                               frame='icrs'
                             )
moving_star_coords

In [ ]:
my_query = f"""
SELECT TOP 100
source_id, ra, dec, parallax, parallax_over_error, pmra, pmdec, radial_velocity
FROM gaiadr3.gaia_source_lite
WHERE DISTANCE( POINT({moving_star_coords.ra.degree}, {moving_star_coords.dec.degree}), POINT(ra, dec) ) < 0.5
AND parallax_over_error > 20
AND radial_velocity IS NOT NULL
ORDER BY pmra
"""

In [ ]:
print(my_query)

In [ ]:
my_job_query = Gaia.launch_job(my_query)

In [ ]:
print(my_job_query)

In [ ]:
target_table = my_job_query.get_results()

In [ ]:
target_table[0:2]

In [ ]:
target_table['Distance'] = target_table['parallax'].to(u.parsec, equivalencies=u.parallax())

In [ ]:
target_table[0:2]

### Make a plot of the proper motions

In [ ]:
fig, ax = plt.subplot_mosaic(
    '''
    ABB
    ''',
    figsize = (8, 4), 
    constrained_layout = True
)

ax['A'].set_xlabel(f'pmra ({target_table["pmra"].unit})')
ax['A'].set_ylabel(f'pmdec ({target_table["pmdec"].unit})')

ax['A'].scatter(
    target_table['pmra'], target_table['pmdec'],
    color = 'hotpink',
    marker = '*'
)

ax['B'].hist(target_table['radial_velocity'],
           bins = 20,
           histtype = 'stepfilled',
           facecolor = 'MediumOrchid')

ax['B'].set_xlabel('Radial Velocity')
ax['B'].set_ylabel('Number');

## There is one really fast star!

---
## What is the speedy star?

In [ ]:
fast_star = target_table[:1]

In [ ]:
fast_star

In [ ]:
from astroquery.simbad import Simbad

In [ ]:
Simbad.query_objectids(f"Gaia DR3 4472832130942575872")

## This is [Barnard's star](https://en.wikipedia.org/wiki/Barnard%27s_Star)

---
### With all of the this data, we can create a full 3D-Time coordinate

- Since the object is moving we need a `Time()` for the observation

In [ ]:
from astropy.time import Time

In [ ]:
Gaia_obs_time = Time('2022-06-13', format='iso')

In [ ]:
fast_star_coords = SkyCoord(
    ra = fast_star['ra'],
    dec = fast_star['dec'],
    distance = fast_star['Distance'],
    pm_ra_cosdec = fast_star['pmra'],
    pm_dec = fast_star['pmdec'],
    radial_velocity = fast_star['radial_velocity'],
    obstime = Gaia_obs_time,
    frame = 'icrs')

In [ ]:
fast_star_coords

### With full 3D-Time coordinates we can calculate the coordinates at future times

- `.apply_space_motion()`

In [ ]:
future_time_one = Time('2122-06-13', format='iso')

In [ ]:
import warnings
warnings.simplefilter('ignore', UserWarning)

In [ ]:
future_time_one

In [ ]:
future_time_one_coords = fast_star_coords.apply_space_motion(new_obstime = future_time_one)

In [ ]:
future_time_one_coords

In [ ]:
future_time_two = Time('2222-06-13', format='iso')

In [ ]:
future_time_two_coords = fast_star_coords.apply_space_motion(new_obstime = future_time_two)

In [ ]:
future_time_two_coords

## Make a Plot

In [ ]:
fig, ax = plt.subplot_mosaic(
    '''
    ABB
    ''',
    figsize = (10, 6), 
    constrained_layout = True
)

ax['A'].set_xlim(269,270)
ax['A'].set_ylim(4.5,5.5)

ax['A'].set_xlabel(f'RA')
ax['A'].set_ylabel(f'DEC')

#ax['A'].set_xlim(269,270)
ax['B'].set_ylim(1.80,1.832)

ax['B'].set_xlabel(f'Date (Year)')
ax['B'].set_ylabel(f'Distance (lyr)')

my_list = [fast_star_coords, future_time_one_coords, future_time_two_coords]
my_color = ['Crimson', 'Lime', 'Indigo']

for my_index, my_coord in enumerate(my_list):
    
    ax['A'].plot(my_coord.ra.degree, my_coord.dec.degree,
        color = my_color[my_index],
        marker = "*",
        linestyle = "None",
        markersize = 25
    )

    ax['B'].plot(my_coord.obstime.decimalyear, my_coord.distance,
        color = my_color[my_index],
        marker = "*",
        linestyle = "None",
        markersize = 25
    )

---

# Moving Targets (solar system targets)

- You need a `Time` for observation since everything is moving.
- The built-in solar system objects are: `sun`, `mercury`, `venus`, `earth-moon-barycenter`, `earth`, `moon`, `mars`, `jupiter`, `saturn`, `uranus`, `neptune`, `pluto`

In [ ]:
from astropy.coordinates import get_body
from astropy.time import Time

In [ ]:
my_date = Time("2025-02-12 21:40")

In [ ]:
sun_now = get_body('sun', my_date)

sun_now

In [ ]:
sun_now.ra

In [ ]:
sun_now.dec

In [ ]:
moon_now = get_body('moon', my_date)

In [ ]:
moon_now.ra

In [ ]:
moon_now.dec

In [ ]:
sun_now.separation(moon_now)

---

## Palomar Observatory Sky Survey (POSS)

The Palomar Observatory Sky Survey (POSS) was a major astronomical survey, that took almost 2,000 photographic plates of the night sky that was completed by the end of 1958. The photographs were taken with the 48-inch Samuel Oschin telescope at Palomar Observatory.

<p>
    <img src="https://uwashington-astro300.github.io/A300_images/HubbleSchmitt.jpg" width = "200">
</p>

The survey utilized 14-inch square photographic plates, covering about 6° of sky per side. Each region of the sky was photographed twice, once using a red plate, and once with a blue sensitive plate.

POSS was mostly completed just proir to the launch of Sputnik 1 on October 4, 1957, meaning the original POSS prints have NO satellite trails on them.

The POSS was published as a collection of 1,872 photographic negative prints each measuring 14" x 14". The original cost was ~ \\$15,000 plus \\$1,650 for the transparent coordinate overlays.


<p>
    <img src="https://uwashington-astro300.github.io/A300_images/POSS_1.jpg" width = "400">
    <img src="https://uwashington-astro300.github.io/A300_images/POSS_2.jpg" width = "400">
</p>

The [Digitized Sky Survey (DSS)](https://archive.stsci.edu/cgi-bin/dss_form) is a digitized version of the Palomar and UK Schmidt surveys, produced by the Space Telescope Science Institute.

- The First Generation DSS (DSS1) was published (on 102 CD-ROMs for \\$3,500) in 1994
- The Second Generation DSS (DSS2) was published online in 2006

## Pretty Pictures

- You can use the `plot_finder_image()` function from the `astroplan` package to get a DSS image of your target object.


In [ ]:
target_coords = SkyCoord.from_name('M 104')
target_coords

In [ ]:
fig, ax = plt.subplots(
    figsize = (6, 6), 
    constrained_layout = True
)

ax, hdu = plot_finder_image(
    target_coords, 
    fov_radius= 0.25 * u.deg, 
    ax = ax,
    survey = 'DSS2 Blue'
);

In [ ]:
fig, ax = plt.subplots(
    figsize = (6, 6), 
    constrained_layout = True
)

ax, hdu = plot_finder_image(
    target_coords, 
    fov_radius= 0.5 * u.deg, 
    ax = ax,
    survey = 'DSS2 IR'
);